### Measuring transaction costs and trading volume

In [195]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import scipy.stats as scipy
sys.path.append("utils")
import dynamic_backtesting as db
from compare_strategies import calc_transaction_costs as ctc
plt.style.use('seaborn')
matplotlib.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=["black", "green",  "deepskyblue", "orange","grey", "lime", "purple", "Gold", "brown", "blue", "pink"]) 

In [196]:
assets = {"EEM": "Emerging Markets",
           "IVV": "S&P 500",
           "IEV": "MSCI Europe",
           "IXN": "Global Tech",
           "IYR": "US Real Estate",
           "IXG": "Global Financials",
           "EXI": "Global Industrials",
           "GC=F": "Gold Futures", 
           "BZ=F": "Brent Crude Oil Futures",
           "HYG": "High-Yield Corporate Bonds",
           "TLT": "20+ Year Treasury Bonds"}
tickers = list(assets.keys())
asset_names = list(assets.values())
stocks = tickers[:7]
commodities = tickers[7:9]
bonds = tickers[9:]

In [197]:
models = ['sGARCH10', 'sGARCH11', 'gjrGARCH11']
tuning_gamma_Ds = [1.52e-6, 1.3e-5, 1e-4]

In [198]:
def remove_extreme(num, tolerance):
    if num > tolerance:
        return tolerance
    if num < 0:
        return np.nan
    return num

In [199]:
def calc_TC(model, tuning_gamma_D, assets):
    v_t_s, out_of_sample, in_sample, Omega_ts = db.garch_with_trading_cost(tickers=assets, model_type=model, tuning_gamma_D=tuning_gamma_D)
    out_of_sample = out_of_sample.divide(100)
    TC = ctc(v_t_s, out_of_sample, Omega_ts)
    TC = [remove_extreme(x, 1) for x in TC]
    return TC

In [218]:
def calc_multiple_TC(tickers):
    TCs = []
    for model in models:
        row = []
        for tuning_gamma_D in tuning_gamma_Ds:
            row.append(calc_TC(model, tuning_gamma_D, tickers))
        TCs.append(row)
    TCs_mean = [[np.nanmean(TC) for TC in TC_row] for TC_row in TCs]
    return TCs, TCs_mean

In [ ]:
TCs_all, TCs_all_mean = calc_multiple_TC(tickers)

In [ ]:
TCs_bonds, TCs_bonds_mean = calc_multiple_TC(bonds)

In [ ]:
TCs_stocks, TCs_stocks_mean = calc_multiple_TC(stocks)

In [ ]:
TCs_com, TCs_com_mean = calc_multiple_TC(commodities)

In [256]:
def annualize_TC(TC):
    TC = np.array(TC)*(-1)
    TC += 1
    TC_cum = np.cumprod(TC)

    total_costs = pd.Series(TC_cum).ffill().iloc[-1]
    non_num_nan = len(TC)-np.isnan(TC).sum()
    ann_TC = 1-(total_costs ** (250/non_num_nan))
    return ann_TC

In [260]:
TC_all_ann = [[annualize_TC(TC) for TC in TC_row] for TC_row in TCs_all]

In [ ]:
TC_bonds_ann = [[annualize_TC(TC) for TC in TC_row] for TC_row in TCs_bonds]

In [ ]:
TC_stocks_ann = [[annualize_TC(TC) for TC in TC_row] for TC_row in TCs_stocks]

In [ ]:
TC_com_ann = [[annualize_TC(TC) for TC in TC_row] for TC_row in TCs_com]

### Equal weight costs

In [220]:
def calc_TC_EW():
    TCs = []
    assets = [tickers, bonds, stocks, commodities]
    for model in models:
        row = []
        for asset_list in assets:
            out_of_sample, in_sample, sigmas, residuals, params_dict = db.split_fit_parse(asset_list, start="2008-01-01", end="2021-10-02",
                                                                               number_of_out_of_sample_days=1000, model_type=model)
            Omega_ts = db.calc_Omega_ts(out_of_sample_returns=out_of_sample, in_sample_returns=in_sample,
                             in_sample_sigmas=sigmas, in_sample_residuals=residuals, **params_dict)
            weight_index = in_sample.index[[-1]].union(out_of_sample.index)
            
            p = len(asset_list)
            weights_shape = (1001, p)
            EW = pd.DataFrame(np.full(weights_shape, (1/p)), index=weight_index)
            out_of_sample = out_of_sample.divide(100)
            TC_EW = ctc(weights=EW, returns=out_of_sample, Omega_ts=Omega_ts, portfolio_value=1e9)
            TC_EW = [remove_extreme(x, 1) for x in TC_EW]
            row.append(TC_EW)
        TCs.append(row)
    return TCs

In [221]:
TCs_EW = calc_TC_EW()

['EEM', 'IVV', 'IEV', 'IXN', 'IYR', 'IXG', 'EXI', 'GC=F', 'BZ=F', 'HYG', 'TLT']
['HYG', 'TLT']
['EEM', 'IVV', 'IEV', 'IXN', 'IYR', 'IXG', 'EXI']
['GC=F', 'BZ=F']
['EEM', 'IVV', 'IEV', 'IXN', 'IYR', 'IXG', 'EXI', 'GC=F', 'BZ=F', 'HYG', 'TLT']
['HYG', 'TLT']
['EEM', 'IVV', 'IEV', 'IXN', 'IYR', 'IXG', 'EXI']
['GC=F', 'BZ=F']
['EEM', 'IVV', 'IEV', 'IXN', 'IYR', 'IXG', 'EXI', 'GC=F', 'BZ=F', 'HYG', 'TLT']
['HYG', 'TLT']
['EEM', 'IVV', 'IEV', 'IXN', 'IYR', 'IXG', 'EXI']
['GC=F', 'BZ=F']


In [258]:
TCs_EW_ann = [[annualize_TC(TC) for TC in TC_row] for TC_row in TCs_EW]

In [259]:
TCs_EW_ann

[[0.04359932591944282,
  0.008481584075029525,
  0.013126672368225956,
  0.06357385076324906],
 [0.05020247006939704,
  0.009966997484608497,
  0.014707829180714227,
  0.08225260437137927],
 [0.05237439341486372,
  0.011340533154792332,
  0.014989338255203766,
  0.0778527615750314]]